In [1]:
#load packages
library(ggplot2)
library(plyr)
library(tidyverse)
library(grid)
library(gridExtra)
library(ggrepel)
library(RColorBrewer)
library(glue)
library(cowplot)
library(UpSetR)
library(pheatmap)
library(vcfR)
library(igraph)
library(ggpubr)
library(readr)
library(VennDiagram)

Warning message:
“package ‘ggplot2’ was built under R version 4.0.2”
Warning message:
“package ‘tidyverse’ was built under R version 4.0.2”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1
✔ purrr   0.3.4     

Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘readr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
Warning message:
“package ‘forcats’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::id()  

In [2]:
ntlist <- c('A','T','C','G','-')

In [3]:
project_folder <- '/Users/roderae/National Institutes of Health/Ghedin, Elodie (NIH NIAID) [E] - LAB_STUFF/allison/projects/COV2-SNV'
mva_folder <- glue("{project_folder}/synthetic_data/flu")

In [108]:
setwd(mva_folder)
# grab files from each caller folder
vcfs.t <- Sys.glob("./timo/*.vcf")
vcfs.hc <- Sys.glob("./hc/*.vcf")
vcfs.i <- Sys.glob("./ivar/*.vcf")
vcfs.fb <- Sys.glob("./freebayes/*.vcf") 
vcfs.v <- Sys.glob("./varscan/*.vcf")
vcfs.m <- Sys.glob("./mutect2/*unfiltered.vcf")
vcfs.l <- Sys.glob("./lofreq/*.vcf")

# make a single list for callers that can be loaded together
vcf_lists <- as.list(c(vcfs.hc,vcfs.i))

#### Lofreq VCF files  

lofreq uses an outdated VCF format and thus cannot be loaded using VCF R

In [109]:
# make a data frame to hold lofreq vcf info 
vcf.lofreq <- data.frame()

# for each file, read in the table
# add a header row
# separate the info by semi-colon so it can be extracted
# get rid of definitions so data can be used for calculations
# make separate columns for tool and file name
for (filename in vcfs.l) {
    # read in a file
    vcf <- read.table(filename,stringsAsFactors = F)
    
    if(nrow(vcf) > 0) {
        names(vcf) <- c("chrom","ntpos","ID","refnt","varnt","qual","filter","info")
        vcf <- vcf %>% separate(info, sep = ';', into = c("DP","AF","SB","DP4"))
        vcf$DP <- gsub("DP=","",vcf$DP)
        vcf$AF <- gsub("AF=","",vcf$AF)
        vcf$SB <- gsub("SB=","",vcf$SB)
        vcf$DP4 <- gsub("DP4=","",vcf$DP4)
        vcf$source_ID <- filename
    
        # separate tool and file columns
        vcf <- separate(vcf, source_ID, sep = "/", 
                        into = c("dot","tool","file")) %>%
                        select(-c(dot))

        # get source_ID
        vcf <- separate(vcf, file, sep = ".fas", 
                        into = c("source_ID","rest")) %>%
                        select(-c(rest))
    
        # round allele frequency to 4 digits
        vcf$DP = as.numeric(vcf$DP)
        vcf$AF = as.numeric(vcf$AF)
        vcf$AF <- round(vcf$AF, digits = 4)

        # select desired columns and name appropriately
        vcf <- vcf %>% select("tool","source_ID","chrom","ntpos","refnt","varnt","AF","DP")
        names(vcf) <- c("tool","source_ID","segment","ntpos","refnt","varnt","varfreq","totalcount")
    
        # add to the lofreq data frame
        vcf.lofreq <- rbind(vcf.lofreq, vcf)
    }
    
}

#### Varscan VCF files

In [110]:
# make data frame to hold varscan vcf files
vcf.varscan <- data.frame()

# for each varscan vcf file
for (filename in vcfs.v) {
    print(filename)
    # read in a file
    # convert to usable data frame
    # set source_ID column
    vcf <- read.vcfR(filename, verbose = FALSE)
    vcf_df <- vcfR2tidy(vcf,single_frame = TRUE)
    vcf_df <- as.data.frame(vcf_df[[1]])
    
    if(nrow(vcf_df) > 0) {
        vcf_df$source_ID <- filename

        # add to varscan data frame
        vcf.varscan <- rbind(vcf.varscan, vcf_df)
    }
    
}

# select desired columns
vcf.varscan <- vcf.varscan %>% select(source_ID, CHROM, POS, REF, ALT, QUAL, FILTER, gt_AD, ADP, gt_AD, gt_DP)

# convert column class to numeric where applicable
vcf.varscan$gt_DP = as.numeric(vcf.varscan$gt_DP)
vcf.varscan$ADP = as.numeric(vcf.varscan$ADP)
# calculate frequency using AD and ADP
vcf.varscan <- vcf.varscan %>%
    mutate(FREQ = gt_AD/ADP)
vcf.varscan$FREQ <- round(vcf.varscan$FREQ, digits = 4)

# separate tool and file columns
vcf.varscan <- separate(vcf.varscan, source_ID, sep = "/", 
               into = c("dot","tool","file")) %>%
                select(-c(dot))

# get source_ID
vcf.varscan <- separate(vcf.varscan, file, sep = ".fas", 
               into = c("source_ID","rest")) %>%
                select(-c(rest))

# select necessary columns
vcf.varscan <- vcf.varscan %>% select(tool, source_ID, CHROM, POS, REF, ALT, FREQ, ADP)

# set names of columns to match other tools
names(vcf.varscan) <- c("tool","source_ID","segment","ntpos","refnt","varnt","varfreq","totalcount")

[1] "./varscan/000000000-K4LKV_l01_SynCon_Mix01_rep1.fas_BWA_varscan_standard.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element SDP

Extracting gt element DP

Extracting gt element RD

Extracting gt element AD

Extracting gt element FREQ

Extracting gt element PVAL

Extracting gt element RBQ

Extracting gt element ABQ

Extracting gt element RDF

Extracting gt element RDR

Extracting gt element ADF

Extracting gt element ADR



[1] "./varscan/000000000-K4LKV_l01_SynCon_Mix01_rep2.fas_BWA_varscan_standard.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element SDP

Extracting gt element DP

Extracting gt element RD

Extracting gt element AD

Extracting gt element FREQ

Extracting gt element PVAL

Extracting gt element RBQ

Extracting gt element ABQ

Extracting gt element RDF

Extracting gt element RDR

Extracting gt element ADF

Extracting gt element ADR



[1] "./varscan/000000000-K4LKV_l01_SynCon_Mix02_rep1.fas_BWA_varscan_standard.vcf"


...

Extracting gt element GT

Extracting gt element GQ

Extracting gt element SDP

Extracting gt element DP

Extracting gt element RD

Extracting gt element AD

Extracting gt element FREQ

Extracting gt element PVAL

Extracting gt element RBQ

Extracting gt element ABQ

Extracting gt element RDF

Extracting gt element RDR

Extracting gt element ADF

Extracting gt element ADR



[1] "./varscan/000000000-K4LKV_l01_SynCon_Mix44_rep1.fas_BWA_varscan_standard.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element SDP

Extracting gt element DP

Extracting gt element RD

Extracting gt element AD

Extracting gt element FREQ

Extracting gt element PVAL

Extracting gt element RBQ

Extracting gt element ABQ

Extracting gt element RDF

Extracting gt element RDR

Extracting gt element ADF

Extracting gt element ADR



[1] "./varscan/000000000-K4LKV_l01_SynCon_Mix44_rep2.fas_BWA_varscan_standard.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element SDP

Extracting gt element DP

Extracting gt element RD

Extracting gt element AD

Extracting gt element FREQ

Extracting gt element PVAL

Extracting gt element RBQ

Extracting gt element ABQ

Extracting gt element RDF

Extracting gt element RDR

Extracting gt element ADF

Extracting gt element ADR



#### Mutect2 VCF files

In [111]:
# create data frame to hold mutect vcf files
vcf.mutect <- data.frame()

# for each mutect2 vcf file
for (filename in vcfs.m) {
    print(filename)
    # read in a file
    vcf <- read.vcfR(filename, verbose = FALSE)
    
    # convert to a standard dataframe
    vcf_df <- vcfR2tidy(vcf,single_frame = TRUE)
    # extract the data frame from the list
    vcf_df <- as.data.frame(vcf_df[[1]])
    
    if (nrow(vcf_df) > 0) {

        # separate multiallelic positions from single allele positions
        # multi allelic positions will not be just A, C, T, or G
        multi <- vcf_df %>% filter(!ALT %in% ntlist)
        single <- vcf_df %>% filter(ALT %in% ntlist)

        # select desired columns and get rid of indels
        single <- single %>% select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, gt_AD, gt_DP) %>% filter(REF %in% ntlist)

        # use allele depth column to calculate variant frequency for single allele positions
        single <- separate(single, gt_AD, sep = ',', into = c("major_dp","minor_dp"))
        single$minor_dp <- as.numeric(single$minor_dp)
        # variant freq is equal to number of reads with the alt nucleotide over the total number of reads
        single <- single %>%
                    mutate(AF = minor_dp/gt_DP)
        single$AF <- as.numeric(single$AF)
        # round decimals to 4 digits
        single$AF <- round(single$AF, digits = 4)
        single <- single %>% select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, AF, gt_DP)
    
        vcf_df <- single
    
        # detect just rows where the alt allele positions contains a comma 
        # indels were not considered for this study
        # select necessary columns and filter out indels
        multi <- multi %>% filter(str_detect(multi$ALT, ",") == TRUE) %>% 
                           select(c(1:10),"gt_AD","gt_DP") %>% 
                           filter(REF %in% ntlist)

        if(nrow(multi) > 0) {
        
            # separate the allele depth column into the ref value, the depth of the first allele and the depth of the second allele
            multi <- separate(multi, gt_AD, sep = ',', into = c("refAF", "adp1", "adp2"))
            # convert all to numeric values
            multi$refAF <- as.numeric(multi$refAF)
            multi$adp1 <- as.numeric(multi$adp1)
            multi$adp2 <- as.numeric(multi$adp2)
            # calculate allele frequency by dividing the depth of each allele by the total depth at a given position
            multi <- multi %>% mutate(AF1 = adp1/gt_DP)
            multi <- multi %>% mutate(AF2 = adp2/gt_DP)
            # convert the allele frequencies to numeric values
            multi$AF1 <- as.numeric(multi$AF1)
            multi$AF2 <- as.numeric(multi$AF2)

            # separate the allele into nucleotide for the first and nucleotide for the second
            multi <- separate(multi, ALT, sep = ",", 
                           into = c("ALT1","ALT2"))
    
            # add a new row for each second allele where the first allele frequency goes with the first allele and second freq with the second allele
            # select desired columns
            multi <- multi %>% pivot_longer(cols = starts_with("ALT"),
                                        names_to = "var",
                                        values_to = "ALT") %>% 
                           mutate(AF=ifelse(var == "ALT1", AF1, AF2)) %>%
                           select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, AF, gt_DP)
        
            # combine the multi allele positions back with the single allele positons
            vcf_df <- rbind(single, multi)
        
        }

    vcf_df$source_ID <- filename
    
    # add to mutect data frame
    vcf.mutect <- rbind(vcf.mutect, vcf_df)
    
    }
}

# separate tool and file columns
vcf.mutect <- separate(vcf.mutect, source_ID, sep = "/", 
               into = c("dot","tool","file")) %>%
                select(-c(dot))

# get source_ID
vcf.mutect <- separate(vcf.mutect, file, sep = ".fas", 
               into = c("source_ID","rest")) %>%
                select(-c(rest))

# change class of columns where necessary
vcf.mutect$gt_DP = as.numeric(vcf.mutect$gt_DP)
vcf.mutect$AF = as.numeric(vcf.mutect$AF)
vcf.mutect$AF <- round(vcf.mutect$AF, digits = 4)

# select necessary columns
vcf.mutect <- vcf.mutect %>% select(tool, source_ID, CHROM, POS, REF, ALT, AF, gt_DP)

# set names of columns to match other tools
names(vcf.mutect) <- c("tool","source_ID","segment","ntpos","refnt","varnt","varfreq","totalcount")

[1] "./mutect2/000000000-K4LKV_l01_SynCon_Mix01_rep1.fas_BWA_mutect2_standard_unfiltered.vcf"


Extracting gt element AD

Extracting gt element AF

Extracting gt element DP

Extracting gt element F1R2

Extracting gt element F2R1

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PGT

Extracting gt element PID

Extracting gt element PL

Extracting gt element PS

Extracting gt element SB



[1] "./mutect2/000000000-K4LKV_l01_SynCon_Mix01_rep2.fas_BWA_mutect2_standard_unfiltered.vcf"


Extracting gt element AD

Extracting gt element AF

Extracting gt element DP

Extracting gt element F1R2

Extracting gt element F2R1

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PGT

Extracting gt element PID

Extracting gt element PL

Extracting gt element PS

Extracting gt element SB



[1] "./mutect2/000000000-K4LKV_l01_SynCon_Mix02_rep1.fas_BWA_mutect2_standard_unfiltered.vcf"


...

Extracting gt element AD

Extracting gt element AF

Extracting gt element DP

Extracting gt element F1R2

Extracting gt element F2R1

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PGT

Extracting gt element PID

Extracting gt element PL

Extracting gt element PS

Extracting gt element SB



[1] "./mutect2/000000000-K4LKV_l01_SynCon_Mix44_rep1.fas_BWA_mutect2_standard_unfiltered.vcf"


Extracting gt element AD

Extracting gt element AF

Extracting gt element DP

Extracting gt element F1R2

Extracting gt element F2R1

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PGT

Extracting gt element PID

Extracting gt element PL

Extracting gt element PS

Extracting gt element SB



[1] "./mutect2/000000000-K4LKV_l01_SynCon_Mix44_rep2.fas_BWA_mutect2_standard_unfiltered.vcf"


Extracting gt element AD

Extracting gt element AF

Extracting gt element DP

Extracting gt element F1R2

Extracting gt element F2R1

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PGT

Extracting gt element PID

Extracting gt element PL

Extracting gt element PS

Extracting gt element SB



### Timo VCF files

In [112]:
# set up dataframe to hold vcf files for timo, haplotype caller and ivar
vcf.all.t <- data.frame()

for (filename in vcfs.t) {    
    print(filename)
    # read in a file
    vcf <- read.vcfR(filename, verbose = FALSE)
    
    # convert to a standard dataframe
    vcf_df <- vcfR2tidy(vcf,single_frame = TRUE)
    # extract the data frame from the list
    vcf_df <- as.data.frame(vcf_df[[1]])
    
    if (nrow(vcf_df) > 0) {

        # separate multiallelic positions from single allele positions
        # multi allelic positions will not be just A, C, T, or G
        multi <- vcf_df %>% filter(!ALT %in% ntlist)
        single <- vcf_df %>% filter(ALT %in% ntlist)
    
        vcf_df <- single
    
        # detect just rows where the alt allele positions contains a comma 
        # indels were not considered for this study
        # select necessary columns and filter out indels
        multi <- multi %>% filter(str_detect(multi$ALT, ",") == TRUE) %>% 
                           filter(REF %in% ntlist)

        if(nrow(multi) > 0) {
        
            multi <- distinct(multi)
    
            # separate alleles
            multi <- separate(multi, ALT, sep = ",", 
                           into = c("ALT1","ALT2"))
            # separate allele frequencies
            multi <- separate(multi, AF, sep = ",",
                           into = c("AF1", "AF2"))
    
            # add a new row for each second allele
            multi <- multi %>% pivot_longer(cols = starts_with("ALT"),
                                    names_to = "var",
                                    values_to = "ALT") %>% 
                       mutate(AF=ifelse(var == "ALT1", AF1, AF2)) %>%
                       select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, AF, Indiv, gt_GT, gt_AD, gt_DP, gt_GQ, gt_GT_alleles)
    
            vcf_df <- rbind(single, multi)
        
        }
        # make source ID column
        vcf_df$source_ID <- filename
    
        # select desired columns in order
        vcf_df <- vcf_df %>% select(source_ID, CHROM, POS, REF, ALT, QUAL, FILTER, AF, gt_DP)
    
        # add to larger data frame
        vcf.all.t <- rbind(vcf.all.t, vcf_df)

    
    }
}

# separate tool and file columns
vcf.all.t <- separate(vcf.all.t, source_ID, sep = "/", 
               into = c("dot","tool","file")) %>%
                select(-c(dot))

# get source_ID
vcf.all.t <- separate(vcf.all.t, file, sep = ".fas", 
               into = c("source_ID","rest")) %>%
                select(-c(rest))

# change class of columns where necessary
vcf.all.t$gt_DP = as.numeric(vcf.all.t$gt_DP)
vcf.all.t$AF = as.numeric(vcf.all.t$AF)
vcf.all.t$AF <- round(vcf.all.t$AF, digits = 4)

# select necessary columns
vcf.all.t <- vcf.all.t %>% select(tool, source_ID, CHROM, POS, REF, ALT, AF, gt_DP)

# set names of columns to match other tools
names(vcf.all.t) <- c("tool","source_ID","segment","ntpos","refnt","varnt","varfreq","totalcount")

[1] "./timo/000000000-K4LKV_l01_SynCon_Mix01_rep1.fas_BWA_timo_standard_no-binom-check.vcf"


Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ



[1] "./timo/000000000-K4LKV_l01_SynCon_Mix01_rep2.fas_BWA_timo_standard_no-binom-check.vcf"


Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ



[1] "./timo/000000000-K4LKV_l01_SynCon_Mix02_rep1.fas_BWA_timo_standard_no-binom-check.vcf"


...

Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ



[1] "./timo/000000000-K4LKV_l01_SynCon_Mix44_rep1.fas_BWA_timo_standard_no-binom-check.vcf"


Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ



[1] "./timo/000000000-K4LKV_l01_SynCon_Mix44_rep2.fas_BWA_timo_standard_no-binom-check.vcf"


Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ



### Freebayes VCF files

In [113]:
# set up dataframe to hold vcf files for timo, haplotype caller and ivar
vcf.all.fb <- data.frame()

for (filename in vcfs.fb) {
    
    print(filename)
    # read in a file
    vcf <- read.vcfR(filename, verbose = FALSE)

    vcf_df <- vcfR2tidy(vcf,single_frame = TRUE)
    vcf_df <- as.data.frame(vcf_df[[1]])
    
    if(nrow(vcf_df) > 0) {
    
        multi <- vcf_df %>% filter(!ALT %in% ntlist)
        single <- vcf_df %>% filter(ALT %in% ntlist)
        
        single <- single %>% select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, gt_AD, gt_DP)

        # use allele depth column to calculate variant frequency
        single <- separate(single, gt_AD, sep = ',', into = c("major_dp","minor_dp"))
        single$minor_dp <- as.numeric(single$minor_dp)
        # variant freq is equal to number of reads with the alt nucleotide over the total number of reads
        single <- single %>%
                mutate(AF = minor_dp/gt_DP)
        single$AF <- as.numeric(single$AF)
        # round decimals to 4 digits
        single$AF <- round(single$AF, digits = 4)
        single <- single %>% select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, AF, gt_DP)
        
        vcf_df <- single
        
        if(nrow(multi) > 0) {

            multi <- multi %>% filter(str_detect(multi$ALT, ",") == TRUE) %>% select(c(1:10),"RO","AO","gt_AD","gt_DP")

            multi <- separate(multi, gt_AD, sep = ',', into = c("refAF", "adp1", "adp2"))
            multi$refAF <- as.numeric(multi$refAF)
            multi$adp1 <- as.numeric(multi$adp1)
            multi$adp2 <- as.numeric(multi$adp2)
            multi <- multi %>% mutate(AF1 = adp1/gt_DP)
            multi <- multi %>% mutate(AF2 = adp2/gt_DP)
            multi$AF1 <- as.numeric(multi$AF1)
            multi$AF2 <- as.numeric(multi$AF2)

            multi <- separate(multi, ALT, sep = ",", 
                       into = c("ALT1","ALT2"))

            multi <- multi %>% pivot_longer(cols = starts_with("ALT"),
                                            names_to = "var",
                                            values_to = "ALT") %>% 
                                mutate(AF=ifelse(var == "ALT1", AF1, AF2)) %>%
                                select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, AF, gt_DP)

            
    
            vcf_df <- rbind(single, multi)
            
        }
    
         # make source ID column
        vcf_df$source_ID <- filename
    
        vcf.all.fb <- rbind(vcf.all.fb, vcf_df)
        
    }
    
}

# separate tool and file columns
vcf.all.fb <- separate(vcf.all.fb, source_ID, sep = "/", 
               into = c("dot","tool","file")) %>%
                select(-c(dot))

# get source_ID
vcf.all.fb <- separate(vcf.all.fb, file, sep = ".fas", 
               into = c("source_ID","rest")) %>%
                select(-c(rest))

# change class of columns where necessary
vcf.all.fb$gt_DP = as.numeric(vcf.all.fb$gt_DP)
vcf.all.fb$AF = as.numeric(vcf.all.fb$AF)
vcf.all.fb$AF <- round(vcf.all.fb$AF, digits = 4)

# select necessary columns
vcf.all.fb <- vcf.all.fb %>% select(tool, source_ID, CHROM, POS, REF, ALT, AF, gt_DP)

# set names of columns to match other tools
names(vcf.all.fb) <- c("tool","source_ID","segment","ntpos","refnt","varnt","varfreq","totalcount")

[1] "./freebayes/000000000-K4LKV_l01_SynCon_Mix01_rep1.fas_BWA_freebayes_standard.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element GL

Extracting gt element DP

Extracting gt element AD

Extracting gt element RO

Extracting gt element QR

Extracting gt element AO

Extracting gt element QA

Extracting gt element MIN_DP



[1] "./freebayes/000000000-K4LKV_l01_SynCon_Mix01_rep2.fas_BWA_freebayes_standard.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element GL

Extracting gt element DP

Extracting gt element AD

Extracting gt element RO

Extracting gt element QR

Extracting gt element AO

Extracting gt element QA

Extracting gt element MIN_DP



[1] "./freebayes/000000000-K4LKV_l01_SynCon_Mix02_rep1.fas_BWA_freebayes_standard.vcf"


...

Extracting gt element GT

Extracting gt element GQ

Extracting gt element GL

Extracting gt element DP

Extracting gt element AD

Extracting gt element RO

Extracting gt element QR

Extracting gt element AO

Extracting gt element QA

Extracting gt element MIN_DP



[1] "./freebayes/000000000-K4LKV_l01_SynCon_Mix44_rep1.fas_BWA_freebayes_standard.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element GL

Extracting gt element DP

Extracting gt element AD

Extracting gt element RO

Extracting gt element QR

Extracting gt element AO

Extracting gt element QA

Extracting gt element MIN_DP



[1] "./freebayes/000000000-K4LKV_l01_SynCon_Mix44_rep2.fas_BWA_freebayes_standard.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element GL

Extracting gt element DP

Extracting gt element AD

Extracting gt element RO

Extracting gt element QR

Extracting gt element AO

Extracting gt element QA

Extracting gt element MIN_DP



#### All other VCF files - ivar and haplotype caller

In [114]:
# set up dataframe to hold vcf files for timo, haplotype caller and ivar
vcf.all1 <- data.frame()

for (filename in vcf_lists) {
    # read in a file
    vcf <- read.vcfR(filename, verbose = FALSE)
    # convert to a usable data frame
    vcf_df <- vcfR2tidy(vcf,single_frame = TRUE)
    vcf_df <- as.data.frame(vcf_df[[1]])
    
    if(nrow(vcf_df) > 0) {
        # make source ID column
        vcf_df$source_ID <- filename
    
        # select desired columns in order
        vcf_df <- vcf_df %>% select(source_ID, CHROM, POS, REF, ALT, QUAL, FILTER, AF, gt_AD, gt_DP)
    
        # add to larger data frame
        vcf.all1 <- rbind(vcf.all1, vcf_df)
    }
    
}

# separate tool and file columns
vcf.all1 <- separate(vcf.all1, source_ID, sep = "/", 
               into = c("dot","tool","file")) %>%
                select(-c(dot))

# get source_ID
vcf.all1 <- separate(vcf.all1, file, sep = ".fas", 
               into = c("source_ID","rest")) %>%
                select(-c(rest))

# change class of columns where necessary
vcf.all1$gt_DP = as.numeric(vcf.all1$gt_DP)
vcf.all1$AF = as.numeric(vcf.all1$AF)
vcf.all1$AF <- round(vcf.all1$AF, digits = 4)

# set names to match other data frames
names(vcf.all1) <- c("tool","source_ID","segment","ntpos","refnt","varnt","qual","filter","varfreq","allele_depth","totalcount")

# use allele depth column to calculate variant frequency
vcf.all1 <- separate(vcf.all1, allele_depth, sep = ',', into = c("major_dp","minor_dp"))
vcf.all1$minor_dp <- as.numeric(vcf.all1$minor_dp)
# variant freq is equal to number of reads with the alt nucleotide over the total number of reads
vcf.all1 <- vcf.all1 %>%
    mutate(AF2 = minor_dp/totalcount)
vcf.all1$AF2 <- as.numeric(vcf.all1$AF2)
# round decimals to 4 digits
vcf.all1$AF2 <- round(vcf.all1$AF2, digits = 4)
# replace varfreq column with AF2 value
vcf.all1 <- vcf.all1 %>% mutate(varfreq = AF2)
# select columns to match other data sets
vcf.all1 <- vcf.all1 %>% select(tool, source_ID, segment, ntpos, refnt, varnt, varfreq, totalcount)

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting g

In [115]:
head(vcf.all1)
head(vcf.all.fb)
head(vcf.all.t)
head(vcf.mutect)
head(vcf.varscan)
head(vcf.lofreq)

,tool,source_ID,segment,ntpos,refnt,varnt,varfreq,totalcount
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>
1,hc,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,43,G,A,0.0049,1228
2,hc,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,236,T,C,0.0035,2024
3,hc,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,298,T,C,0.4216,2244
4,hc,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,333,T,C,0.4262,2419
5,hc,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,515,G,A,0.3842,2691
6,hc,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,516,A,G,0.4213,2846


,tool,source_ID,segment,ntpos,refnt,varnt,varfreq,totalcount
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>
1,freebayes,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,4,A,G,0.0269,1376
2,freebayes,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,298,T,C,0.4266,3835
3,freebayes,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,333,T,C,0.4501,4481
4,freebayes,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,621,T,C,0.4110,5778
5,freebayes,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,796,G,T,0.3953,5280
6,freebayes,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,831,T,C,0.4100,4583


,tool,source_ID,segment,ntpos,refnt,varnt,varfreq,totalcount
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>
1,timo,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,4,A,G,0.0269,1376
2,timo,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,298,T,C,0.4265,3838
3,timo,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,333,T,C,0.4503,4484
4,timo,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,515,G,A,0.3846,4561
5,timo,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,516,A,G,0.3881,4677
6,timo,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,596,C,A,0.0353,5929


,tool,source_ID,segment,ntpos,refnt,varnt,varfreq,totalcount
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>
1,mutect2,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,43,G,A,0.0052,1159
2,mutect2,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,224,C,T,0.0027,1880
3,mutect2,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,277,T,C,0.0028,2140
4,mutect2,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,298,T,C,0.4253,2229
5,mutect2,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,333,T,C,0.4295,2405
6,mutect2,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,596,C,A,0.0305,3118


,tool,source_ID,segment,ntpos,refnt,varnt,varfreq,totalcount
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>
1,varscan,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,4,A,G,0.0401,548
2,varscan,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,298,T,C,0.4208,3660
3,varscan,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,333,T,C,0.4410,4252
4,varscan,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,515,G,A,0.3414,4110
5,varscan,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,516,A,G,0.3368,4166
6,varscan,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,596,C,A,0.0348,5749


,tool,source_ID,segment,ntpos,refnt,varnt,varfreq,totalcount
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>
1,lofreq,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,236,T,C,0.0028,2844
2,lofreq,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,298,T,C,0.4263,3807
3,lofreq,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,333,T,C,0.4505,4442
4,lofreq,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,462,T,C,0.0029,3752
5,lofreq,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,515,G,A,0.3840,4550
6,lofreq,000000000-K4LKV_l01_SynCon_Mix01_rep1,HA,516,A,G,0.3871,4671


In [116]:
# combine all the VCF files from each tools
vcf.all <- do.call("rbind", list(vcf.all1, vcf.all.fb, vcf.all.t, vcf.mutect, vcf.varscan, vcf.lofreq))

In [117]:
nrow(vcf.all)

[1] 34586

In [118]:
ntlist <- c('A','T','C','G')
vcf.all <- vcf.all %>% filter(refnt %in% ntlist & varnt %in% ntlist)
nrow(vcf.all)

[1] 34311

In [119]:
write_excel_csv(x = vcf.all, file = glue("{mva_folder}/flu.synthetic.vcfs.csv"))